Aprendizaje Reforzado

**Aprendizaje Reforzado**

> En este ejercicio se tiene una simulación de la problemática abordada por un taxi común, se basa principalmente en los movimientos que debe realizar este para recoger y transportar un pasajero a su lugar de destino. Como en un entorno real, existirán movimientos prohibidos, rutas más largas y rutas más cortas de un punto a otro, a través de las cuales el taxi deberá llevar al pasajero, obteniendo recompensas de acuerdo a la eficiencia y los movimientos realizados dentro del espacio determinado.












---



Se valida que se tenga la libreria instalada

In [1]:
#!pip install cmake 'gym[atari]' scipy

Se importa la libreria gym la cual posee varios escenarios y problematicas que pueden aplicarse a traves de aprendizaje reforzado y se llama el ambiente del problema que se va a trabajar, en este caso el entorno del taxi. Posteriormente se grafica el estado inicial de este.

In [2]:
import gym

taxi = gym.make("Taxi-v3").env

taxi.render()

/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't 

ResetNeeded: ignored

 
*   El cuadro amarillo representa el taxi, el cual es amarillo cuando está vacio y verde cuando lleva un pasajero.
*   El simbolo pipe (|) representa paredes que el taxi no puede cruzar.
*   R, G, Y, B son las posibles localizaciones donde se puede recoger y dejar un pasajero. La letra azul representa el actual lugar de recoger el pasajero. La letra morada será el destino.




***Problema Markoviano***


*   El agente sería nuestro vehículo
*  Los estados son cada uno de los lugares en donde el taxi puede ir, además de las únicas posiciones en donde puede recoger o dejar el pasajero. En total, son 500 posibles estados que vienen dados por: la grilla de la ciudad de 5x5 (25 posibles localizaciones del taxi), a esto se  multiplica los 4 lugares de destino (R, G, Y, B o en coordenadas: [(0,0), (0,4), (4,0),(4,3)) teniendo 5x5x4, y finalmente los 4 lugares de recogida del pasajero (que son los mismos puntos R, G, Y, B) al cual hay que sumarle la posibilidad de que esté el pasajero en el carro (4+1), teniendo entonces 5x5x4x5. Eso nos da los 500 posibles estados en los que el taxi se puede encontrar.


* El conjunto de acciones que puede ejecutar el agente serían 6:
> 1. Ir al norte
> 2. Ir al sur
> 3. Ir al este
> 4. Ir al oeste
> 5. Recoger al pasajero
> 6. Dejar al pasajero

* La probabilidad de transición es igual para todas las transiciones entre estados, pues en el escenario evaluado el vehículo puede dirigirse a cualquier dirección sin ningún tipo de sesgo.

* Como el "agente" va a aprender a manejar el taxi experimentando en el ambiente, las recompensas y las penalidades a considerar son:
> * El agente debe recibir una recompensa positiva alta llevar al pasajero a su destino de manera exitosa, porque este comportamiento es muy deseado (+20 por dejar de manera exitosa al pasajero).
> * El agente debe ser penalizado si lleva o recoge al pasajero a un lugar incorrecto (-10 por dejar o recoger al pasajero en lugares no permitidos).
> * El agente debe tener una recompensa levemente negativa por no llegar a la localización en cada paso (-1 por cada paso). 
> * El agente cuando realiza una accion que no se puede realizar en ciertos estados, porque hay "paredes", se hará una penalización de -1 cada que se choque con una pared.





Posteriormente se hace un reinicio del estado inicial del taxi para que este se ubique de manera aleatoria en otra posicion y estado.

In [ ]:
taxi.reset() # se reinicia el ambiente a un nuevo estado de manera aleatoria
taxi.render()

print("Espacio de acciones {}".format(taxi.action_space))
print("Espacio de estados {}".format(taxi.observation_space))

Necesitamos una manera de identificar cada estado de una manera unica, asignando un unico numero a cada posible estado y el algoritmo de aprendizaje reforzado aprenderá a tomar una acción con un número de 0 a 5, donde: 

**A. Acciones:**
0 = south;
1 = north;
2 = east;
3 = west;
4 = pickup;
5 = dropoff;

**B. Localización del pasajero:**
0: R;
1: G;
2: Y;
3: B;
4: en el taxi

**C. Destino:**
0: R;
1: G;
2: Y;
3: B


Por ejemplo, supongamos que:

* El taxi está en la posición  (3,1) 
* El pasajero está en la localización Y, que es  2 
* El destino es R, que es  0 

*Nuestro estado será: (3,1,2,0)*
---


Con el método taxi.encode podemos codificar el estado. Miremos:

In [ ]:
estado = taxi.encode(3,1,2,0) # fila taxi, columna taxi, localización pasajero, destino
print("Estado: ", estado)

taxi.s = estado
taxi.render()

Estamos usando las coordenadas de la imagen para generar un número de  0−499  correspondiente a un estado, en este caso es el estado  328 .

Así se puede hacer para todas las posibles combinaciones y cada una tendrá un valor de  0−499  y se podrá hacer de manera manual como ya se vió anteriormente.



Para tener un mayor entendimiento del problema, se ubica el taxi en unas coordenadas especificas y frente a un estado deseado de la ubicacion del pasajero y su destino, se prueban diferentes ubicaciones para entender el problema

In [ ]:
estado = taxi.encode(0, 1, 4, 1) # (ubicacion fila, ubicacion columna, ubicacion pasajero, destino del pasajero)
print("Estado:", estado)

taxi.s = estado
taxi.render()

**La tabla de recompensas**

Para tener un mayor entendimiento del problema, se ubica el taxi en unas coordenadas especificas y frente a un estado deseado de la ubicacion del pasajero y su destino, se prueban diferentes ubicaciones para entender el problema

Cuando el ambiente "Taxi" es creado, también hay una tabla de recompensas que se crea. Se puede pensar en esta como una matriz que tiene el número de estados como filas y el número de acciones como columnas:

Dado que cada estado está en la matriz, podemos ver los valores de recompensa por defecto asignados a nuestra imagen.

Para el analisis de esta problematica, se decide elegir un estado igual a 37 para ubicar el taxi, el pasajero y su destino

In [ ]:
taxi.P[37]

Este diccionario es de la forma: 

```
{accion0: [(probabilidad, siguiente estado, recompensa, ¿listo?)], 
 accion1: [(probabilidad, siguiente estado, recompensa, ¿listo?)], 
 ... , 
 accion5:[(probabilidad, siguiente estado, recompensa, ¿listo?)]}
```

Anotaciones: 
- 0 a 5 corresponde a las acciones (south, north, east, west, pickup, dropoff) que el taxi puede realizar en nuestro actual estado (en la imagen)
- En este ambiente, la probabilidad (```probabilidad```) de que algo pase siempre es $1.0$
- Siguiente estado (```siguiente estado```) es el estado al que pasaría si se toma la acción.
- Para este estado, todas las acciones de movimiento tienen una recompensa (```recompensa```) de $-1$, y las acciones de  pickup/dropoff de $-10$. Si estuvieramos en un estado en el destino del pasajero, la  acción de dropoff  aparecería una recompensa de $20$ para esta acción. 
- ```listo``` es usado para decirno si el pasajero fue dejado en su destino exitosamente. Por cada pasajero dejado en su destino de manera exitosa se completa un episodio. 

Notemos que si el agente explora la acción 2 en este estado, va a "chocar" contra una pared, el algoritmo hace que sea imposible moverse a través de una pared, por lo que si se elije esta acción se irán acumulando penalizaciones de $-1$, lo que afectar a la **recompensa a largo plazo**



Una vez observados los posibles movimientos, se procede a fijar esa posición como el estado de origen en donde incluso se tiene definido hacia donde se quiere llevar el pasajero y donde se debe recoger.

## Resolviendo el ambiente **SIN** aprendizaje reforzado

Miremos que pasaría si se quiere resolver el problema a partir de la "fuerza bruta"

Como tenemos la tabla de recompensas para cada estado (```P```), se puede poner a navegar al taxi solo con eso. 

Se crea un ciclo infinito que corra hasta que un pasajero sea dejado a su destino de manera correcta (esto es un **episodio**), o en otras palabras, que se obtenga la recompensa de $20$. El método ```taxi.action_space.sample()``` automaticamente selecciona una acción aleatoria del conjunto de posibles acciones. 

Miremos qué pasa: 

In [ ]:
taxi.s = 37  # se fija el estado deseado

pasos = 0
castigos, recompensa = 0, 0

frames = [] # for animation

done = False

while not done:
    accion = taxi.action_space.sample()
    estado, recompensa, done, info = taxi.step(accion)

    if recompensa == -10:
        castigos += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': taxi.render(mode='ansi'),
        'estado': estado,
        'acción': accion,
        'recompensa': recompensa
        }
    )

    pasos += 1
    
    
print("Pasos tomados: {}".format(pasos))
print("Castigos incurridos: {}".format(castigos))

Se observa que se necesitó mas de mil pasos para ir de la posición inicial a recoger el pasajero y dejarlo en su lugar de destino, de los cuales se cometio un buen numero de fallas. A continuación se visualiza cada uno de los movimientos realizados por el taxi (Estos datos pueden variar según la posición en la que se ubique el taxi cuando este se resetea la posición inicial)

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Pasos: {i + 1}")
        print(f"estado: {frame['estado']}")
        print(f"accion: {frame['acción']}")
        print(f"recompensa: {frame['recompensa']}")
        sleep(.1)
        
print_frames(frames)

### Implementando *Q-learning*

A partir de este momento se crea el modelo de entrenamiento reforzado para que el sistema aprenda y sea capaz de cometer el menor numero de faltas y optimizar los pasos necesarios.

In [ ]:
import numpy as np
q_table = np.zeros([taxi.observation_space.n, taxi.action_space.n])

Ahora podemos crear un algoritmo de entrenamiento que actualice esta matriz-Q a medida que el agente explora el ambiente en varios episodios.

El modelo consiste en definir los hiperparametros del proceso, es decir los valores de alpha (tasa de aprendizaje), gamma (factor de descuento), y epsilon (medida de mejora en el aprendizaje). Estos hiperparametros son de gran importancia dentro del modelo de aprendizaje.

---

En la primera parte de while not done, se decide si se quieren tomar acciones aleatorias (explorar) o si se quieren explotar los valores-Q ya aprendidos. Esto se hace utilizando el valor  ϵ  y comparandolo con un valor aleatorio random.uniform(0,1).

El modelo de entrenamiento indica que si el numero aleatorio es menor al epsilon, entonces la accion es cualquier (explorar), de lo contrario se determina la proxima accion segun q usando los parametros alpha y gamma (explotar)



In [ ]:
%%time
import random
from IPython.display import clear_output

# Hiperparametros
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# Para mostrar las metricas
Pasos_Totales = []
Fallas_totales = []

for i in range(1, 100001):
    estado = taxi.reset()

    pasos, castigos, recompensa, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            accion = taxi.action_space.sample() 
        else:
            accion = np.argmax(q_table[estado]) 

        prox_estado, recompensa, done, info = taxi.step(accion) 
        
        valor_anterior = q_table[estado, accion]
        max_proximo = np.max(q_table[prox_estado])
        
        nuevo_valor = (1 - alpha) * valor_anterior + alpha * (recompensa + gamma * max_proximo)
        q_table[estado, accion] = nuevo_valor

        if recompensa == -10:
            castigos += 1

        estado = prox_estado
        pasos += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Escenario: {i}")

print("Entrenamiento finalizado.\n")

Se visualiza el vector de entrenamiento para el estado deseado

Ya tenemos la matriz-Q después de varios episodios, miremos los valores-Q para nuestra imágen. <br>

Recordemos que el episodio era el número $37$. Y que: <br>
**Acciones**
- 0 = south
- 1 = north
- 2 = east
- 3 = west
- 4 = pickup
- 5 = dropoff


In [ ]:
q_table[37]

Posicion 0 es la mejor


Se evalua de nuevo el taxi despues del entrenamiento reforzado

In [ ]:
Pasos_Totales, Fallas_totales = 0, 0
episodios = 100

for _ in range(episodios):
    estado = taxi.reset()
    pasos, fallas, recompensa = 0, 0, 0
    
    done = False
    
    while not done:
        accion = np.argmax(q_table[estado])
        estado, recompensa, done, info = taxi.step(accion)

        if recompensa == -10:
            fallas += 1

        pasos += 1

    Fallas_totales += fallas
    Pasos_Totales += pasos

print(f"Resultados despues de {episodios} episodios:")
print(f"Promedio de pasos por episodio: {Pasos_Totales / episodios}")
print(f"Promedio fallas por episodio: {Fallas_totales / episodios}")

Se observa que en 100 episodios no se cometio ni una falla por lo que el taxi quedó con un buen ajuste de entrenamiento

Se puede evaluar como cambia el modelo si se modifican los hiperparametros, para ver la sensibilidad que este tiene, primero se realiza con alpha que es la tasa de aprendizaje, cambiando esta de 0.1 a 0.4

In [ ]:
Pasos_Totales, Fallas_totales = 0, 0
episodios = 100

for _ in range(episodios):
    estado = taxi.reset()
    pasos, fallas, recompensa = 0, 0, 0
    
    done = False
    
    while not done:
        accion = np.argmax(q_table[estado])
        estado, recompensa, done, info = taxi.step(accion)

        if recompensa == -10:
            fallas += 1

        pasos += 1

    Fallas_totales += fallas
    Pasos_Totales += pasos

print(f"Resultados despues de {episodios} episodios:")
print(f"Promedio de pasos por episodio: {Pasos_Totales / episodios}")
print(f"Promedio fallas por episodio: {Fallas_totales / episodios}")

Cuando se observa el resultado, se encuentra que el promedio incrementa un poco, y tiempo de procesamiento es el mismo, por lo que no hay mucha sensibilidad de alpha en el modelo de aprendizaje, luego se retorna alpha a 0.1 y se cambia gamma de 0.6 a 0.4 la tasa de descuento

In [ ]:
%%time

q_table = np.zeros([taxi.observation_space.n, taxi.action_space.n])


import random
from IPython.display import clear_output

# Hiperparametros
alpha = 0.1
gamma = 0.4
epsilon = 0.1

# Para mostrar las metricas
Pasos_Totales = []
Fallas_totales = []

for i in range(1, 100001):
    estado = taxi.reset()

    pasos, castigos, recompensa, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            accion = taxi.action_space.sample() 
        else:
            accion = np.argmax(q_table[estado]) 

        prox_estado, recompensa, done, info = taxi.step(accion) 
        
        valor_anterior = q_table[estado, accion]
        max_proximo = np.max(q_table[prox_estado])
        
        nuevo_valor = (1 - alpha) * valor_anterior + alpha * (recompensa + gamma * max_proximo)
        q_table[estado, accion] = nuevo_valor

        if recompensa == -10:
            castigos += 1

        estado = prox_estado
        pasos += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Escenario: {i}")

print("Entrenamiento finalizado.\n")

In [ ]:
Pasos_Totales, Fallas_totales = 0, 0
episodios = 100

for _ in range(episodios):
    estado = taxi.reset()
    pasos, fallas, recompensa = 0, 0, 0
    
    done = False
    
    while not done:
        accion = np.argmax(q_table[estado])
        estado, recompensa, done, info = taxi.step(accion)

        if recompensa == -10:
            fallas += 1

        pasos += 1

    Fallas_totales += fallas
    Pasos_Totales += pasos

print(f"Resultados despues de {episodios} episodios:")
print(f"Promedio de pasos por episodio: {Pasos_Totales / episodios}")
print(f"Promedio fallas por episodio: {Fallas_totales / episodios}")

Cambiando gama, aunque requiere un poco mas de tiempo, se requieren menos pasos por episodio para evaluar el modelo y aun asi no ocurre fallas, por ultimo  se cambia epsilon a 0.5 retornando gamma a su valor de 0.6

In [ ]:
%%time

q_table = np.zeros([taxi.observation_space.n, taxi.action_space.n])


import random
from IPython.display import clear_output

# Hyperparametros
alpha = 0.1
gamma = 0.6
epsilon = 0.5

# Para mostrar las metricas
Pasos_Totales = []
Fallas_totales = []

for i in range(1, 100001):
    estado = taxi.reset()

    pasos, castigos, recompensa, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            accion = taxi.action_space.sample() 
        else:
            accion = np.argmax(q_table[estado]) 

        prox_estado, recompensa, done, info = taxi.step(accion) 
        
        valor_anterior = q_table[estado, accion]
        max_proximo = np.max(q_table[prox_estado])
        
        nuevo_valor = (1 - alpha) * valor_anterior + alpha * (recompensa + gamma * max_proximo)
        q_table[estado, accion] = nuevo_valor

        if recompensa == -10:
            castigos += 1

        estado = prox_estado
        pasos += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Escenario: {i}")

print("Entrenamiento finalizado.\n")

In [ ]:
Pasos_Totales, Fallas_totales = 0, 0
episodios = 100

for _ in range(episodios):
    estado = taxi.reset()
    pasos, fallas, recompensa = 0, 0, 0
    
    done = False
    
    while not done:
        accion = np.argmax(q_table[estado])
        estado, recompensa, done, info = taxi.step(accion)

        if recompensa == -10:
            fallas += 1

        pasos += 1

    Fallas_totales += fallas
    Pasos_Totales += pasos

print(f"Resultados despues de {episodios} episodios:")
print(f"Promedio de pasos por episodio: {Pasos_Totales / episodios}")
print(f"Promedio fallas por episodio: {Fallas_totales / episodios}")

## Tareas

1) Que pasa cuando se modifica epsilon?, es mas rapido?? menos exigente? se requieren mas paso?? hagan una tabla de valores de epsilo para ese analisis

2) Cuales son los 3 hiperparametros en donde sea optimo el tiempo de maquina y el numero de pasos requeridos para encontrar la solución.

3) Que pasa si se cambia el estado inicial?